In [ ]:
!pip install --upgrade openpyxl

In [ ]:
import pandas as pd
# Leer el archivo Excel en un DataFrame
df = pd.read_excel('Personal Injury matters by Attorney Export.xlsx')

In [ ]:
df.head()

,Lead Attorney,Case Type,Status,Matter,Current Step,SOL,Specials,Branch Office,Case Manager,RIC,Limits,Intake Date,Disbursed
0,BETH,Auto Accident Chicago,Close File – Lien Outstanding,Salazar (Noelia) v. Griffin,2022-02-11,2022-04-09,215591.28,Disparti Platinum,ALEXIS,State Farm Insurance*,NaN,2020-06-17,NaN
1,BETH,Auto Accident Chicago,Close File – Lien Outstanding,Frison (Tasha) v. Saldana,2022-10-20,2024-08-05,24080.00,Disparti Gold,NICOLE,State Farm Insurance*,NaN,2022-08-15,NaN
2,ADAM,Auto Accident Chicago,Negotiation,Parker (Joe) v. Jackson,2023-02-17,2023-11-04,75423.62,Disparti Platinum,IRIS,Geico Insurance Company*,NaN,2021-11-15,NaN
3,BETH,Auto Accident Florida,Litigation,Lawson (Kelsey) v. Egleston,2023-03-20,2025-07-29,56001.08,Disparti Platinum,NICOLE,Allstate Insurance Company*,250/500,2021-08-02,NaN
4,BETH,Auto Accident Florida,Litigation,Scott (Katherine) v. Haupert,2023-03-20,2026-02-23,53482.70,Disparti Platinum,NICOLE,NaN,NaN,2022-03-02,NaN


In [ ]:
# Actualizada
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle, Alignment, Font

# Filtrar por 'Status'
status_filter = ['Negotiation', 'Demand Sent', 'Ready for Demand', 'Settlement']

# Obtener la lista única de "Lead Attorney"
lead_attorneys = df['Lead Attorney'].unique()

# Definir estilos
date_style = NamedStyle(name="datetime", number_format="M/D/YYYY", alignment=Alignment(horizontal="left"))
currency_style = NamedStyle(name="currency", number_format="$#,##0.00;($#,##0.00)", alignment=Alignment(horizontal="left"))
default_font = Font(name='Times New Roman', size=11)
bold_font = Font(name='Times New Roman', size=11, bold=True)

In [ ]:
lead_attorneys

array(['BETH', 'ADAM', 'AARON', 'RAPHAEL.FORBES', 'JGOLDBERG', 'KEVIN'],
      dtype=object)

In [ ]:
# Iterar sobre cada "Lead Attorney"
for attorney in lead_attorneys:
    # Filtrar el DataFrame por "Lead Attorney" y "Status"
    filtered_df = df[(df['Lead Attorney'] == attorney) & (df['Status'].isin(status_filter))]

    if not filtered_df.empty:
        # Guardar el DataFrame filtrado en un nuevo archivo Excel
        output_file = f'{attorney}.xlsx'
        filtered_df.to_excel(output_file, sheet_name=attorney,index=False)

        # Cargar el archivo Excel para formatearlo
        wb = load_workbook(output_file)
        ws = wb.active

        # Aplicar formatos a las celdas según el tipo de valor
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                #Formatear Fuentes
                if row[0].row == 1:  # Primera fila
                    cell.font = bold_font
                else:
                    cell.font = default_font
                # Formatear fechas
                if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                    cell.style = date_style
                    cell.font = default_font
                # Formatear valores numéricos
                elif isinstance(cell.value, (int, float)):
                    cell.style = currency_style
                    cell.font = default_font

        # Agregar hoja "Silver - RFD"
        silver_rfd_df = df[(df['Lead Attorney'] == attorney) &
                           (df['Status'] == 'Ready for Demand') &
                           (df['Branch Office'] == 'Disparti Silver')]

        if not silver_rfd_df.empty:
            ws_silver_rfd = wb.create_sheet(title="Silver - RFD")
            for r_idx, row in enumerate(dataframe_to_rows(silver_rfd_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_silver_rfd.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_silver_rfd.iter_rows(min_row=2, max_row=ws_silver_rfd.max_row, min_col=1, max_col=ws_silver_rfd.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Agregar hoja "Silver - Demand Sent"
        silver_demand_sent_df = df[(df['Lead Attorney'] == attorney) &
                           (df['Status'] == 'Demand Sent') &
                           (df['Branch Office'] == 'Disparti Silver')]

        if not silver_demand_sent_df.empty:
            ws_silver_demand_sent = wb.create_sheet(title="Silver - Demand Sent")
            for r_idx, row in enumerate(dataframe_to_rows(silver_demand_sent_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_silver_demand_sent.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_silver_demand_sent.iter_rows(min_row=2, max_row=ws_silver_demand_sent.max_row, min_col=1, max_col=ws_silver_demand_sent.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Agregar hoja "Silver - Negotiation"
        silver_negotiation_df = df[(df['Lead Attorney'] == attorney) &
                           (df['Status'] == 'Negotiation') &
                           (df['Branch Office'] == 'Disparti Silver')]

        if not silver_negotiation_df.empty:
            ws_silver_negotiation = wb.create_sheet(title="Silver - Negotiation")
            for r_idx, row in enumerate(dataframe_to_rows(silver_negotiation_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_silver_negotiation.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_silver_negotiation.iter_rows(min_row=2, max_row=ws_silver_negotiation.max_row, min_col=1, max_col=ws_silver_negotiation.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Agregar hoja "Gold - RFD"
        gold_rfd_df = df[(df['Lead Attorney'] == attorney) &
                           (df['Status'] == 'Ready for Demand') &
                           (df['Branch Office'] == 'Disparti Gold')]

        if not gold_rfd_df.empty:
            ws_gold_rfd = wb.create_sheet(title="Gold - RFD")
            for r_idx, row in enumerate(dataframe_to_rows(gold_rfd_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_gold_rfd.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_gold_rfd.iter_rows(min_row=2, max_row=ws_gold_rfd.max_row, min_col=1, max_col=ws_gold_rfd.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Agregar hoja "Gold - Demand Sent"
        gold_demand_sent_df = df[(df['Lead Attorney'] == attorney) &
                                 (df['Status'] == 'Demand Sent') &
                                 (df['Branch Office'] == 'Disparti Gold')]

        if not gold_demand_sent_df.empty:
            ws_gold_demand_sent = wb.create_sheet(title="Gold - Demand Sent")
            for r_idx, row in enumerate(dataframe_to_rows(gold_demand_sent_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_gold_demand_sent.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_gold_demand_sent.iter_rows(min_row=2, max_row=ws_gold_demand_sent.max_row, min_col=1, max_col=ws_gold_demand_sent.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Agregar hoja "Gold - Negotiation"
        gold_negotiation_df = df[(df['Lead Attorney'] == attorney) &
                                 (df['Status'] == 'Negotiation') &
                                 (df['Branch Office'] == 'Disparti Gold')]

        if not gold_negotiation_df.empty:
            ws_gold_negotiation = wb.create_sheet(title="Gold - Negotiation")
            for r_idx, row in enumerate(dataframe_to_rows(gold_negotiation_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_gold_negotiation.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_gold_negotiation.iter_rows(min_row=2, max_row=ws_gold_negotiation.max_row, min_col=1, max_col=ws_gold_negotiation.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Agregar hoja "Platinum - RFD"
        platinum_rfd_df = df[(df['Lead Attorney'] == attorney) &
                           (df['Status'] == 'Ready for Demand') &
                           (df['Branch Office'] == 'Disparti Platinum')]

        if not platinum_rfd_df.empty:
            ws_platinum_rfd = wb.create_sheet(title="Platinum - RFD")
            for r_idx, row in enumerate(dataframe_to_rows(platinum_rfd_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_platinum_rfd.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_platinum_rfd.iter_rows(min_row=2, max_row=ws_platinum_rfd.max_row, min_col=1, max_col=ws_platinum_rfd.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Agregar hoja "Platinum - Demand Sent"
        platinum_demand_sent_df = df[(df['Lead Attorney'] == attorney) &
                                 (df['Status'] == 'Demand Sent') &
                                 (df['Branch Office'] == 'Disparti Platinum')]

        if not platinum_demand_sent_df.empty:
            ws_platinum_demand_sent = wb.create_sheet(title="Platinum - Demand Sent")
            for r_idx, row in enumerate(dataframe_to_rows(platinum_demand_sent_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_platinum_demand_sent.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_platinum_demand_sent.iter_rows(min_row=2, max_row=ws_platinum_demand_sent.max_row, min_col=1, max_col=ws_platinum_demand_sent.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Agregar hoja "Platinum - Negotiation"
        platinum_negotiation_df = df[(df['Lead Attorney'] == attorney) &
                                 (df['Status'] == 'Negotiation') &
                                 (df['Branch Office'] == 'Disparti Platinum')]

        if not platinum_negotiation_df.empty:
            ws_platinum_negotiation = wb.create_sheet(title="Platinum - Negotiation")
            for r_idx, row in enumerate(dataframe_to_rows(platinum_negotiation_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_platinum_negotiation.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_platinum_negotiation.iter_rows(min_row=2, max_row=ws_platinum_negotiation.max_row, min_col=1, max_col=ws_platinum_negotiation.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Agregar hoja "Settlement"
        settlement_df = df[(df['Lead Attorney'] == attorney) &
                                 (df['Status'] == 'Settlement')]

        if not settlement_df.empty:
            ws_settlement = wb.create_sheet(title="Settlement")
            for r_idx, row in enumerate(dataframe_to_rows(settlement_df, index=False, header=True)):
                for c_idx, value in enumerate(row):
                    cell = ws_settlement.cell(row=r_idx+1, column=c_idx+1, value=value)
                    cell.font = bold_font if r_idx == 0 else default_font
            # Aplicar formato a la nueva hoja
            for row in ws_settlement.iter_rows(min_row=2, max_row=ws_settlement.max_row, min_col=1, max_col=ws_settlement.max_column):
                for cell in row:
                    cell.font = default_font
                    if isinstance(cell.value, (pd.Timestamp, pd.to_datetime('today').date().__class__)):
                        cell.style = date_style
                        cell.font = default_font
                    elif isinstance(cell.value, (int, float)):
                        cell.style = currency_style
                        cell.font = default_font

        # Guardar el archivo con los formatos aplicados
        wb.save(output_file)